In [1]:
pip install -U FlagEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.1/866.1 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 34.9 MB/s eta 0:00:00
  Created wheel for FlagEmbedding: filename=FlagEmbedding-1.3.5-py3-none-any.whl size=233746 sha256=7bf72a846e853eb429682b251f2bdf9d3bd9f6414417bc4a3bef5c8fed0a6b08
  Stored in directory: /root/.cache/pip/wheels/b2/1f/f6/78f862bb80cb959cc9960b7c4e2d1f702b1bc0e79d19b5f124
  Created wheel for warc3-wet-clueweb09: filename=warc3_wet_clueweb09-0.2.5-py3-none-any.whl size=18919 sha256=995b1339479292fa74e3b27fa2570c35c5277159bdfa95c5b6f6d63dbcf0f9f6
  Stored in directory: /root/.ca

In [3]:
import numpy as np
import pandas as pd
from FlagEmbedding import FlagReranker
import os
from tqdm import tqdm
import re

# Load BGE reranker model
reranker = FlagReranker('BAAI/bge-reranker-base', use_fp16=True)

def get_relevance_score(question, answer):
    """
    Get relevance score using BGE reranker
    """
    try:
        score = reranker.compute_score([[question, answer]])
        return score[0] if isinstance(score, list) else score
    except Exception as e:
        print(f"Error computing score: {e}")
        return 0.0
def analyze_and_optimize_answer(question, answer):


    parts = answer.split('\n\n')
    intro = []
    points = []
    conclusion = []

    if parts[-2].lower().startswith("confidence score"):
      conclusion.append(parts[-2]+parts[-1])
      parts = parts[:-2]

    elif parts[-1].lower().startswith("confidence score"):
      conclusion.append(parts[-1])
      parts = parts[:-1]

    if not (parts[0].lower().startswith("- ") or parts[0].lower().startswith("*") or parts[0].lower().startswith("1")):
      intro.append(parts[0])
      parts = parts[1:]
    else:
      intro.append("")

    points = parts
    full_answer = '\n\n'.join(intro + points)
    baseline_score = get_relevance_score(question, full_answer)

    impacts={}



    scored_points = []  # will contain (impact_value, point_text)
    for i in range(len(points)):

        remaining_points = [points[j] for j in range(len(points)) if j != i]

        if isinstance(intro, str):
            intro_list = [intro]
        elif isinstance(intro, list):
            intro_list = intro
        else:
            intro_list = [str(intro)]


        modified_answer = '\n\n'.join(intro_list + remaining_points)


        modified_score = get_relevance_score(question, modified_answer)

        point_text = points[i]

        impact =  baseline_score - modified_score

        point_title=points[i][0:100]
        impact_desc=""
        if impact > 0:
            impact_desc=f"Removing reduces relevance score from {baseline_score} to {modified_score} in total= {impact:.4f}"
        elif impact < 0:
            impact_desc=f"Removing increases relevance score from {baseline_score} to {modified_score} in total= {impact:.4f}"
        else:
            impact_desc=f"No impact on relevance score for Point ({point_title})"
        impacts[point_title] = impact_desc
        scored_points.append((impact, point_text))  # store actual point

    # Sort points from highest to lowest impact (most relevant first)
    reordered_points = [pt for _, pt in sorted(scored_points,reverse=True)]

    # Build reranked answer using intro + reordered points
    reranked_answer = '\n\n'.join(intro + reordered_points)

    return {
        'detailed_impacts': impacts,
        'reranked_answer': reranked_answer,
        'similarity_score': baseline_score,

    }





def process_csv_add_optimization(input_path, output_path, batch_size=10):
    df = pd.read_csv(input_path)
    # df=df[0:15]
    results = []

    if os.path.exists(output_path):
        os.remove(output_path)

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing answers with BGE reranker"):
        try:
            question = row.get('question', '')
            answer = row.get('Answer', '')
            print("processing row number",idx)


            if pd.isna(question) or pd.isna(answer) or not question or not answer:
                result_row = row.to_dict()
                result_row['detailed_impacts'] = 'Missing question or answer'
                result_row['reranked_answer'] = answer
                result_row['is_answer_changed'] = False
                result_row['similarity_score'] = 0.0
                results.append(result_row)
                continue


            analysis = analyze_and_optimize_answer(question, answer)
            result_row = row.to_dict()
            result_row['detailed_impacts'] = analysis['detailed_impacts']
            result_row['reranked_answer'] = analysis['reranked_answer']
            result_row['is_answer_changed'] = analysis['reranked_answer'].strip() != answer.strip()
            result_row['similarity_score'] = analysis['similarity_score']
            results.append(result_row)

        except Exception as e:
            print(f"Error in row {idx}: {e}")
            result_row = row.to_dict()
            result_row['detailed_impacts'] = f'Error: {str(e)}'
            result_row['reranked_answer'] = f'Error: {str(e)}'
            result_row['is_answer_changed'] = f'Error: {str(e)}'
            result_row['similarity_score'] = f'Error: {str(e)}'

            results.append(result_row)


        if (idx + 1) % batch_size == 0:
            batch_df = pd.DataFrame(results)
            write_mode = 'w' if idx + 1 == batch_size else 'a'
            batch_df.to_csv(output_path, mode=write_mode, header=(write_mode == 'w'), index=False)
            results = []
            print(f"Saved up to row {idx + 1}...")


    if results:
        batch_df = pd.DataFrame(results)
        batch_df.to_csv(output_path, mode='a', header=not os.path.exists(output_path), index=False)

    print(f"\n✅ Complete! Output saved to: {output_path}")
    final_df = pd.read_csv(output_path)
    print(f"✅ Total rows processed: {len(final_df)}")

if __name__ == "__main__":

    process_csv_add_optimization('/content/answer_gemini.csv', '/content/answer_relevancy_bge_gemini.csv', batch_size=5)


    # CSV processing
    # process_csv_add_optimization("/content/claims.csv", "/content/answer_relevancy_bge_July7.csv", batch_size=10)

Processing answers with BGE reranker:   0%|          | 0/2000 [00:00<?, ?it/s]

processing row number 0


You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Processing answers with BGE reranker:   0%|          | 1/2000 [00:00<20:16,  1.64it/s]

processing row number 1


Processing answers with BGE reranker:   0%|          | 4/2000 [00:01<06:34,  5.07it/s]

...

processing row number 1991
processing row number 1992
processing row number 1993
processing row number 1994


Processing answers with BGE reranker: 100%|█████████▉| 1995/2000 [03:17<00:00,  5.65it/s]

Saved up to row 1995...
processing row number 1995
processing row number 1996
processing row number 1997


Processing answers with BGE reranker: 100%|██████████| 2000/2000 [03:18<00:00, 10.09it/s]


processing row number 1998
processing row number 1999
Saved up to row 2000...

✅ Complete! Output saved to: /content/answer_relevancy_bge_gemini.csv
✅ Total rows processed: 2000
